In [6]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import tensorflow_probability as tfp
import scipy
import numpy as np
import keras

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
scale = 1
means = [5, 15, 25, 35]
num_locations_S = 4
num_components_C = num_locations_S

# simulate data for each location
data_distributions = [scipy.stats.norm(loc=mean, scale=scale) for mean in means]

# Use t-3 data points as features
num_features_per_location_H = 3
num_datapoints_T = 100
# need to draw T+H so every point has enough history
data_samples_to_draw = num_features_per_location_H+num_datapoints_T
# eventually we'll flatten data
num_features_total_F = num_features_per_location_H*num_locations_S
input_shape = (num_datapoints_T, num_features_total_F)

seed=360

data_HT_S = np.zeros((data_samples_to_draw, num_locations_S))
for s, dist in enumerate(data_distributions):
    random_state = np.random.RandomState(seed+s*1000)
    data_HT_S[:, s] = dist.rvs(size=data_samples_to_draw, random_state=random_state)

# Reshape so that previous H samples are features for time t
X_THS = np.array([data_HT_S[t:num_features_per_location_H+t,:] for t in range(num_datapoints_T)], dtype=np.float32)
y_TS = np.array([data_HT_S[num_features_per_location_H+t, :] for t in range(num_datapoints_T)], dtype=np.float32)

# reshape inputs so that data is 2D: sample-by-features
X_TF = np.reshape(X_THS, (num_datapoints_T, -1))
assert(X_TF.shape == input_shape)

In [51]:
class MixtureWeightLayer(keras.layers.Layer):
    """Dumb layer that just returns mixture weights
    Constrained to unit norm
    """
    def __init__(self, num_locations, num_components=2, **kwargs):
        super().__init__(**kwargs)
        self.w = self.add_weight(shape=(num_locations, num_components ),
            initializer="uniform",
            trainable=True,
        )
        
        self.softmax = keras.layers.Softmax(axis=1)

    def call(self, inputs):
        return self.softmax(self.w)

In [52]:
# build layers
inputs = keras.Input(shape=input_shape)
component_layers = [keras.layers.Dense(1, activation='softplus') for _ in range(num_components_C)]
mixture_weight_layer = MixtureWeightLayer(num_locations_S, num_components_C)
mixture_weights = mixture_weight_layer([0])
assert(mixture_weights.shape == (1,num_locations_S, num_components_C))
assert(np.isclose(tf.reduce_sum(mixture_weights, axis=1).numpy(), np.ones(num_locations_S)).all())

# Add a component dimension to outputs of each component model
reshape_layer = keras.layers.Reshape(name='mix_reshape', target_shape=(-1,1))
# Concatenate components along new dimension
concat_layer = keras.layers.Concatenate(name='mix_concat',axis=-1)

# get tfp mixture model
mixture_distribution_layer = tfp.layers.DistributionLambda(lambda params: 
        tfp.distributions.MixtureSameFamily(mixture_distribution=
                                                tfp.distributions.Categorical(probs=params[0]),
                                            components_distribution=
                                                tfp.distributions.Normal(loc=params[1],
                                                                        scale=scale,
                                                                        validate_args=True)))


AssertionError: 

In [ ]:
# build model
component_predictions = [component(inputs) for component in component_layers]
combined_components = concat_layer([reshape_layer(member) for member in component_predictions])
output_distribution = mixture_distribution_layer([mixture_weights, combined_components])
model = keras.Model(inputs=inputs,outputs=outputs)

ValueError: Exception encountered when calling layer "distribution_lambda_3" (type DistributionLambda).

Incompatible shapes for broadcasting. Two shapes are compatible if for each dimension pair they are either equal or one of them is 1. Received: (None, 100) and (1, 4).

Call arguments received by layer "distribution_lambda_3" (type DistributionLambda):
  • inputs=['tf.Tensor(shape=(1, 4, 4), dtype=float32)', 'tf.Tensor(shape=(None, 100, 4), dtype=float32)']
  • args=<class 'inspect._empty'>
  • kwargs={'training': 'None'}

In [43]:
combined_components

<KerasTensor: shape=(None, 100, 4) dtype=float32 (created by layer 'mix_concat')>

In [26]:
tf.reduce_sum(mixture_weights, axis=0).numpy()

array([1.0100352, 0.9951776, 1.0068451, 0.9879423], dtype=float32)